In [1]:
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement
import math
import scipy.io as sio
import random
import os
import tensorflow as tf

In [8]:
x_min = -694
x_max = 685.5
y_min = -651
y_max = 638

In [9]:
# grid size 128*128
grid_size = 100
x_step = (x_max - x_min) / grid_size
y_step = (y_max - y_min) / grid_size

grid_x = np.linspace(x_min, x_max, grid_size)
grid_y = np.linspace(y_min, y_max, grid_size)

# create grid
ue_pos_grid = []

for i in range(grid_size):
    for j in range(grid_size):
        x = grid_x[i]
        y = grid_y[j]
        ue_pos_grid.append(f'[{x} {y} 1.5]')

position_file_name = 'Data/UE_loc/UE_pos_building6.txt'
with open(position_file_name, 'w') as file:
    num = 0
    for item in ue_pos_grid:
        file.write(f"{item}\n")
        num += 1

In [10]:
# Load integrated scene
scene = load_scene('building_xml/building6.xml')
scene.frequency = 3.5e9
    
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Create transmitter
tx = Transmitter(name="tx",
                 position=[149.5, -97.1, 26],
                 orientation=[0, 0, 0])

Tx_pos = np.array([149.5, -97.1, 26]).reshape(-1, 3)

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
              position=[147.3, 124, 1.5],
              orientation=[0,0,0])

# Add receiver instance to scene
scene.add(rx)

scene.synthetic_array = True

tx.look_at(rx) # Transmitter points towards receiver

# Open the file and read its contents
with open('Data/UE_loc/UE_pos_building6.txt', 'r') as file:
    lines = file.readlines()

# Initialize an empty list to store the coordinates
ue_pos = []

# Loop through each line in the file
for line in lines:
    # Extract the numbers inside the brackets and convert them to floats
    coord = [float(num) for num in line.strip().strip('[]').split()]
    # Add the coordinate to the list
    ue_pos.append(coord)

# Now, coordinates contains all the coordinates as lists of floats
print(ue_pos)
print(len(ue_pos))

batch_size = len(ue_pos)

scene.remove("rx")
for i in range(batch_size):
    scene.remove(f"rx-{i}")

for i in range(batch_size):
    rx = Receiver(name=f"rx-{i}",
                  position=ue_pos[i], # Random position sampled from coverage map
                  )
    scene.add(rx)
scene.preview(show_devices=True)


[[-694.0, -651.0, 1.5], [-694.0, -637.9797979797979, 1.5], [-694.0, -624.959595959596, 1.5], [-694.0, -611.939393939394, 1.5], [-694.0, -598.9191919191919, 1.5], [-694.0, -585.8989898989898, 1.5], [-694.0, -572.8787878787879, 1.5], [-694.0, -559.8585858585859, 1.5], [-694.0, -546.8383838383838, 1.5], [-694.0, -533.8181818181818, 1.5], [-694.0, -520.7979797979798, 1.5], [-694.0, -507.77777777777777, 1.5], [-694.0, -494.75757575757575, 1.5], [-694.0, -481.73737373737373, 1.5], [-694.0, -468.7171717171717, 1.5], [-694.0, -455.6969696969697, 1.5], [-694.0, -442.67676767676767, 1.5], [-694.0, -429.65656565656565, 1.5], [-694.0, -416.6363636363636, 1.5], [-694.0, -403.6161616161616, 1.5], [-694.0, -390.5959595959596, 1.5], [-694.0, -377.57575757575756, 1.5], [-694.0, -364.55555555555554, 1.5], [-694.0, -351.5353535353535, 1.5], [-694.0, -338.5151515151515, 1.5], [-694.0, -325.4949494949495, 1.5], [-694.0, -312.47474747474746, 1.5], [-694.0, -299.45454545454544, 1.5], [-694.0, -286.4343434343

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …